In [1]:
import trimesh as triMesh
from random import randint
from random import uniform
import random
import time


import plotly
plotly.offline.init_notebook_mode()
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.tools import FigureFactory as FF
import numpy as np
import os


import scipy.signal as signal
from scipy.interpolate import interp1d

import matplotlib.pyplot as plt


#### lets define some functions first

In [2]:
# this function takes as input two points and returns the point between them at param t
def interpolatePoint (v1,v2,t):
    # subtract v1-v2
    diffV = [  v1[0] - v2[0]  , v1[1] - v2[1]  , v1[2] - v2[2]  ]
    # multiply output with t
    diffVmul = [ diffV[0] * t , diffV[1] * t  , diffV[2] * t ]
    # add to v2
    final = [ v2[0] + diffVmul[0] , v2[1] + diffVmul[1] , v2[2] + diffVmul[2]  ]
    return final

In [3]:
# this function returns midpoint between 3 given points
def midPoint (v1,v2,v3):
    return [ (v1[0]+v2[0]+v3[0])/3  , (v1[1]+v2[1]+v3[1])/3  ,  (v1[2]+v2[2]+v3[2])/3  ]

In [4]:
# takes a mesh and a random face index
# returns a random point on that face
def getRandomPointOnRandomFace(mesh,randomIndex):
    # get vertices
    v1 = mesh.vertices [ mesh.faces[randomIndex][0] ]
    v2 = mesh.vertices [ mesh.faces[randomIndex][1] ]
    v3 = mesh.vertices [ mesh.faces[randomIndex][2] ]
    # get point in between at random param
    v1New = interpolatePoint (v1,v2,  uniform(0,1)  ) # will not reach 1
    v2New = interpolatePoint (v2,v3,  uniform(0,1)  ) # will not reach 1
    v3New = interpolatePoint (v3,v1,  uniform(0,1)  ) # will not reach 1
    # get midpoint
    output =  midPoint(v1New,v2New,v3New)
    return output

#### lets make the D2 mother function

In [5]:
def getD2(mesh,N):
    D2 = []
    start = time.time()
    #
    for i in range ( N ):
        #
        randomIndex1 =  randint(0, len( mesh.faces ) - 1 )  # range end inclusive
        rand1 = getRandomPointOnRandomFace(mesh,randomIndex1)  
        #
        randomIndex2 =  randint(0, len( mesh.faces ) - 1 )  # range end inclusive
        rand2 = getRandomPointOnRandomFace(mesh,randomIndex2) 
        #
        dist = np.sqrt ( np.power( ( rand1[0] - rand2[0]) , 2 ) + np.power( ( rand1[1] - rand2[1]) , 2 ) + np.power( ( rand1[2] - rand2[2]) , 2 )   )
        D2.append(dist)

    end = time.time()
    print end-start
    #
    return D2

In [6]:
# func to normalize D2 between 0 and 1
def normalize(vector):
    return [ ( vector[i]-min(vector) )/( max(vector)-min(vector) ) for i in range( len(vector) ) ]


####lets run on a bunch of meshes

In [38]:
link = 'D:/Dropbox/DFCI/shape-quant/tumors/nodule_'
files = ['3','6','2046','2050','16309','sphere']
# times to run each param
data=[]
# N = np.power(350,2) ############
mul = 8
Nused = []
for i in iter(files): 
    print i
    # get mesh
    mesh = triMesh.load_mesh(link + i+ '.stl')
    # get N Value
    N = int(len(mesh.faces)*mul)
    Nused.append(str(N) + '_' + str( len(mesh.faces) ) )
    # get D2
    D2 = getD2(mesh,N) 
    # normalize
    D2normalized = normalize(D2)
    # append
    data.append( D2normalized )

3
61.5140001774
6
119.840999842
2046
168.456999779
2050
53.123000145
16309
159.654999971
sphere
46.9719998837


#### now lets plot

In [39]:
# http://help.plot.ly/histogram/

fig = FF.create_distplot(data, ['nodule3_' + Nused[0],'nodule6_'+ Nused[1],'nodule2046_'+ Nused[2],'nodule2050_'+ Nused[3],
                                'nodule16309_' + Nused[4], 'sphere_' + Nused[5]], show_hist=False,show_rug=False,show_curve=True,
                         bin_size=[0.05,0.05,0.05,0.05,0.05,0.05], histnorm='probability')

fig['layout'] = go.Layout(
        title='D2 - # of mesh faces * ' + str(mul),
        xaxis=go.XAxis(   
           title='normalized distance',
            range=[0,1]
        ),
        yaxis=go.YAxis(
            title='probaility (%)',
       ),
    )
      

iplot(fig, image='svg', filename = 'facesx' + str(mul))

In [254]:
# figure out a way to compare these curves
# extract vectors - how to compare distributions
# slice randomly and measure area of crosssection

0.094502794449388985

In [231]:
# this function takes a vector and smoothes it
# it uses Butterworth filter + forward-backward filter + cubic interpolation
# it takes the vector and number of divisions on the x to consider

def smooth(vector,bins):

    # generate the bins
    n, bins, patches = plt.hist(vector,bins)

    myX = bins[:len(bins)-1]
    myY = n

    # BUTTER
    # GET THE WINDOW
    b, a = signal.butter(3, 0.25, analog=False, btype = 'lowpass' , output='ba')
    fitData = signal.filtfilt(b,a,myY)

    # EXTRA STEP NEEDED TO SMOOTH - interpolate (this gives a function)
    f2 = interp1d(myX, fitData, kind='cubic')

    # MAKE A NEW X DOMAIN
    # newX = np.linspace(3,19.8,300)

    return myX,f2(myX)

# plt.plot(myX,myY,'o', ms = 3)
# #plt.plot(myX, fitData)
# plt.plot(newX, f2(newX))
# plt.xlim(0,20)
# plt.ylim(0,40)

# plt.title("Butterworth filter + forward-backward filter + cubic interpolation" )
# plt.xlabel("X dimension (cm)" )
# plt.ylabel("Frequency")

# #plt.plot(myX,myY)
# plt.savefig("fig/09a_histogram-lines-BUTTERandfilt.eps")

In [233]:
# Create a trace
xAndy = smooth(D2,15)


data = [
    
    go.Histogram(
        x=D2,
        name='D2',
        autobinx=True,
#         xbins=dict(
#             start=0,
#             end=max(D2),
#             size=1.1
#         ),
        marker=dict(
            color='rgb(0, 0, 255)'
        ),
        opacity=0.50
    ),
    
    go.Scatter(
    x = xAndy[0],
    y = xAndy[1]
    )
    
    ]

iplot(data)



In [51]:
# http://stackoverflow.com/questions/12141150/from-list-of-integers-get-number-closest-to-a-given-value/12141511#12141511
    
def takeClosest(myList, myNumber):
    """
    Assumes myList is sorted. Returns closest index to myNumber.
    If two numbers are equally close, return the smallest number.
    """
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return myList[0]
    if pos == len(myList):
        return myList[-1]
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
       return pos # after
    else:
       return pos - 1 #before

In [29]:
# get triangle areas and cumilative areas
cumilativeAreaList = np.cumsum(mesh.area_faces)

In [91]:
areaMethod = []
directMethod = []
for i in range(50000):
    areaMethod.append(  takeClosest(cumilativeAreaList,uniform(0, cumilativeAreaList[-1] ))   )
    directMethod.append(  randint(0, len( mesh.faces ) - 1 )   )

In [17]:
histo = []
start = time.time()
for i in range (np.power(400,2) ):
    rand1 = mesh.vertices [  randint(0, len( mesh.vertices ) - 1 )  ]
    rand2 = mesh.vertices [  randint(0, len( mesh.vertices ) - 1 )  ]
    dist = np.sqrt ( np.power( ( rand1[0] - rand2[0]) , 2 ) + np.power( ( rand1[1] - rand2[1]) , 2 ) + np.power( ( rand1[2] - rand2[2]) , 2 )   )
    histo.append(dist)
end = time.time()
print end-start

2.83200001717
